This is mainly to just test RAG solutions for this type of problem. I will maybe try to implement a custom graphrag later.

In [10]:
from sentence_transformers import SentenceTransformer
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.utils import is_flash_attn_2_available

import csv
import pandas as pd

In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"

Creating vector database embeddings

In [16]:
embedding_model = SentenceTransformer('all-MiniLM-L6-v2', device=device)

# Load in sentences
df = pd.read_csv('mathinfo.csv')

# Sentences are encoded/embedded by calling model.encode()
embeddings = embedding_model.encode(df['Description'].tolist(), convert_to_tensor=True, device=device)
df['embedding'] = embeddings.tolist()

loading configuration file config.json from cache at C:\Users\justi\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2\snapshots\c9745ed1d9f207416be6d2e6f8de32d1f16199bf\config.json
Model config BertConfig {
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.50.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file model.safetensors from cache at C:\Users\justi\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2\snapshots\c9745ed1d9f207416be6d2e6f8de32d1f16

In [18]:
embeddings = torch.tensor(df['embedding'].tolist(), dtype=torch.float32, device=device)
embeddings.shape

torch.Size([5, 384])

In [26]:
query = "What is the derivative of x^2?"
print(f"Query: {query}")

query_embedding = embedding_model.encode(query, convert_to_tensor=True, device=device)

dot_scores = torch.nn.functional.cosine_similarity(query_embedding, embeddings)
top_k = 5
top_k_indices = torch.topk(dot_scores, top_k).indices
dot_scores_list = dot_scores.tolist()
print(f"Top {top_k} most similar sentences:")
for i in top_k_indices:
    print(f"Score: {dot_scores_list[i]:.4f}, Operation: {df['Operation'][i.item()]}")

Query: What is the derivative of x^2?
Top 5 most similar sentences:
Score: 0.1576, Operation: Exponentiation
Score: 0.0899, Operation: Multiplication
Score: 0.0881, Operation: Subtraction
Score: 0.0654, Operation: Division
Score: -0.0251, Operation: Addition
